In [ ]:
# instalar Parquet, Pandas e Pyarrow
%!pip install parquet
%!pip install pandas
%!pip install pyarrow

In [98]:
# importar os pacotes necessários
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import time

In [103]:
csv_file_path = '../../databases/csv/PFW_2021_public.csv'
output_dir = 'dataset/compression'
os.makedirs(output_dir, exist_ok=True)

# Obtém o tamanho do arquivo CSV original para usar como base
try:
    tamanho_csv = os.path.getsize(csv_file_path)

except FileNotFoundError:
    print(f"Erro: Arquivo CSV original '{csv_file_path}' não encontrado. A taxa de compressão não será calculada.")
    tamanho_csv = None # Define como None para evitar erros depois

Tamanho do arquivo CSV original: 14704853 bytes


In [108]:
def criar_parquet_com_compressao(csv_path, output_path, compression='snappy', tamanho_csv=None):
    """
    Lê um CSV, converte para Parquet com compressão, mede tempo, tamanho e calcula taxa de redução em relação ao CSV.

    Args:
        csv_path (str): Caminho para o CSV.
        output_path (str): Caminho para o Parquet de saída.
        compression (str, optional): Tipo de compressão. Padrão: 'snappy'.
        tamanho_csv (int, optional): Tamanho do arquivo CSV para cálculo da taxa de redução.
    """
    try:
        df = pd.read_csv(csv_path)
        table = pa.Table.from_pandas(df)

        start_time = time.time()
        pq.write_table(table, output_path, compression=compression)
        end_time = time.time()

        tamanho_arquivo = os.path.getsize(output_path)
        print(f"Arquivo Parquet com compressão {compression if compression else 'sem compressão'} criado em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho do arquivo: {tamanho_arquivo} bytes")

        if tamanho_csv:
            taxa_compressao = (tamanho_csv - tamanho_arquivo) / tamanho_csv * 100
            print(f"Taxa de redução em relação ao CSV: {taxa_compressao:.2f}%") # Mensagem alterada

    except FileNotFoundError:
        print(f"Erro: Arquivo '{csv_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro na conversão para PyArrow: {e}")
    except Exception as e:
        print(f"Ocorreu um erro ao escrever o Parquet: {e}")


def main():

    print(f"Tamanho do arquivo CSV original: {tamanho_csv} bytes")


    output_parquet_path_sem_compressao = os.path.join(output_dir, 'PFW_2021_public_sem_compressao.parquet')
    criar_parquet_com_compressao(csv_file_path, output_parquet_path_sem_compressao, compression=None, tamanho_csv=tamanho_csv)

    output_parquet_path_snappy = os.path.join(output_dir, 'PFW_2021_public_snappy.parquet')
    criar_parquet_com_compressao(csv_file_path, output_parquet_path_snappy, compression='snappy', tamanho_csv=tamanho_csv)

    # output_parquet_path_gzip = os.path.join(output_dir, 'PFW_2021_public_gzip.parquet')
    # criar_parquet_com_compressao(csv_file_path, output_parquet_path_gzip, compression='gzip', tamanho_csv=tamanho_csv)

    # output_parquet_path_brotli = os.path.join(output_dir, 'PFW_2021_public_brotli.parquet')
    # criar_parquet_com_compressao(csv_file_path, output_parquet_path_brotli, compression='brotli', tamanho_csv=tamanho_csv)

    # output_parquet_path_zstd = os.path.join(output_dir, 'PFW_2021_public_zstd.parquet')
    # criar_parquet_com_compressao(csv_file_path, output_parquet_path_zstd, compression='zstd', tamanho_csv=tamanho_csv)



if __name__ == "__main__":
    main()

Tamanho do arquivo CSV original: 14704853 bytes
Arquivo Parquet com compressão sem compressão criado em 0.0601 segundos.
Tamanho do arquivo: 4725428 bytes
Taxa de redução em relação ao CSV: 67.86%
Arquivo Parquet com compressão snappy criado em 0.0687 segundos.
Tamanho do arquivo: 3307142 bytes
Taxa de redução em relação ao CSV: 77.51%


In [ ]:
try:
    # criando o diretório do arquivo titanic.parquet
    df_parquet_path = '../../databases/parquet/PFW_2021_public.parquet'

    # criando o diretório de destino dos arquivos de compressão, caso não exista
    # output_dir = 'dataset/compression'
    # os.makedirs(output_dir, exist_ok=True)

    output_path = 'dataset/compression'
    os.makedirs(output_path, exist_ok=True)

    # ler os dados do arquivo csv para um DataFrame no Pandas
    df_csv = pd.read_csv('../../databases/csv/titanic.csv')

    # Converter para tabela PyArrow
    table = pa.Table.from_pandas(df_csv)

except FileNotFoundError:
    print(f"Erro: Arquivo '{df_parquet_path}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

## Compressão de arquivo Parquet com Snappy

In [ ]:
def comprimir_parquet_snappy(input_path, output_path):
    """
    Comprime um arquivo Parquet usando Snappy.

    Args:
        input_path (str): Caminho para o arquivo Parquet de entrada.
        output_path (str): Caminho para o arquivo Parquet de saída (com compressão Snappy).
    """
    try:
        start_time = time.time()

        # Ler o arquivo Parquet usando PyArrow
        table = pq.read_table(input_path)

        # Escrever o arquivo Parquet com compressão Snappy
        pq.write_table(table, output_path, compression='snappy')

        end_time = time.time()

        tamanho_original = os.path.getsize(input_path)
        tamanho_comprimido = os.path.getsize(output_path)
        taxa_compressao = (tamanho_original - tamanho_comprimido) / tamanho_original * 100

        print(f"Arquivo comprimido com Snappy em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho original: {tamanho_original} bytes")
        print(f"Tamanho comprimido: {tamanho_comprimido} bytes")
        print(f"Taxa de compressão: {taxa_compressao:.2f}%")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
    except Exception as e:
        print(f"Ocorreu um erro durante a compressão: {e}")

def main():
    """Função principal."""
    input_parquet = '../../databases/parquet/PFW_2021_public.parquet'  # Substitua pelo caminho do seu arquivo Parquet
    output_parquet_snappy = 'dataset/PFW_2021_public_snappy.parquet' # Substitua pelo caminho de saida desejado
    output_dir = 'dataset'
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_parquet):
        print(f"Erro: Arquivo Parquet original '{input_parquet}' não encontrado.")
        return

    comprimir_parquet_snappy(input_parquet, output_parquet_snappy)

if __name__ == "__main__":
    main()

## Comparação de compressão em arquivos Parquet - snappy | gzip | brotli | zstd

In [ ]:
try:
    # Definindo as opções de compressão
    compressions = {
        'sem_compressao': None,
        'snappy': 'snappy',
        'gzip': 'gzip',
        'brotli': 'brotli',
        'zstd': 'zstd',
        # 'lzo': 'lzo'
    }

    #
    resultados = {}
    for nome, compressao in compressions.items():
        nome_arquivo = os.path.join(output_dir, f'titanic_{nome}.parquet')
        start_time_escrita = time.time()
        pq.write_table(table, nome_arquivo, compression=compressao)
        end_time_escrita = time.time()
        tamanho_arquivo = os.path.getsize(nome_arquivo)

        start_time_leitura = time.time()
        pd.read_parquet(nome_arquivo)  # Lê o arquivo para medir o tempo de leitura.
        end_time_leitura = time.time()

        resultados[nome] = {
            'tempo_escrita': end_time_escrita - start_time_escrita,
            'tempo_leitura': end_time_leitura - start_time_leitura,
            'tamanho': tamanho_arquivo  # Mantém o tamanho aqui
        }

    print("Resultados da comparação de compressão:\n")
    print("{:<15} {:<15} {:<15} {:<15}".format("Compressão", "Tempo Escrita", "Tempo Leitura", "Tamanho (bytes)")) # Ajuste na ordem dos cabeçalhos
    print("-" * 60)

    for nome, dados in resultados.items():
        print("{:<15} {:<15.4f} {:<15.4f} {:<15}".format( # Ajuste na ordem dos dados
            nome, dados['tempo_escrita'], dados['tempo_leitura'], dados['tamanho']
        ))

except Exception as e:
    print(f"Ocorreu um erro: {e}")

In [ ]:
try:
    # Ler os dados do arquivo CSV para um DataFrame do Pandas
    df = pd.read_csv('../../databases/csv/titanic.csv')

    # Converter o DataFrame do Pandas para uma tabela PyArrow
    table = pa.Table.from_pandas(df)

    # Definir as opções de gravação com compressão Snappy
    parquet_file = os.path.join(output_dir, 'titanic_snappy.parquet')
    start_time = time.time()
    pq.write_table(table, parquet_file, compression='snappy')
    end_time = time.time()
    print(f"Arquivo Parquet com compressão Snappy criado em {end_time - start_time:.4f} segundos.")
    print(f"Tamanho do arquivo: {os.path.getsize(parquet_file)} bytes")

    # Criando o arquivo Parquet sem compressão
    parquet_file_sem_compressao = os.path.join(output_dir, 'titanic_sem_compressao.parquet')
    start_time = time.time()
    pq.write_table(table, parquet_file_sem_compressao)
    end_time = time.time()
    print(f"Arquivo Parquet SEM compressão criado em {end_time - start_time:.4f} segundos.")
    print(f"Tamanho do arquivo: {os.path.getsize(parquet_file_sem_compressao)} bytes")

except FileNotFoundError:
    print("Erro: Arquivo '../../databases/csv/titanic.csv' não encontrado. Certifique-se de que o arquivo existe no diretório correto.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

In [ ]:
try:
    # Criando o arquivo Parquet com compressão Gzip
    parquet_file_gzip = os.path.join(output_dir, 'titanic_gzip.parquet')
    start_time = time.time()
    pq.write_table(table, parquet_file_gzip, compression='gzip')
    end_time = time.time()
    print(f"Arquivo Parquet com compressão Gzip criado em {end_time - start_time:.4f} segundos.")
    print(f"Tamanho do arquivo: {os.path.getsize(parquet_file_gzip)} bytes")

    # Criando o arquivo Parquet com compressão Brotli
    parquet_file_brotli = os.path.join(output_dir, 'titanic_brotli.parquet')
    start_time = time.time()
    pq.write_table(table, parquet_file_brotli, compression='brotli')
    end_time = time.time()
    print(f"Arquivo Parquet com compressão Brotli criado em {end_time - start_time:.4f} segundos.")
    print(f"Tamanho do arquivo: {os.path.getsize(parquet_file_brotli)} bytes")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

In [ ]:
def criar_parquet_com_compressao(input_path, output_path, compression='snappy'):
    """
    Lê um arquivo Parquet, salva uma cópia com compressão especificada e mede o tempo e tamanho.

    Args:
        input_path (str): Caminho para o arquivo Parquet de entrada.
        output_path (str): Caminho para o arquivo Parquet de saída.
        compression (str, optional): Tipo de compressão ('snappy', 'gzip', 'brotli', 'zstd', None). Padrão é 'snappy'.
    """
    try:
        start_time = time.time()

        # Ler o Parquet original usando PyArrow diretamente para evitar conversão desnecessária
        table = pq.read_table(input_path)

        pq.write_table(table, output_path, compression=compression)

        end_time = time.time()

        tamanho_arquivo = os.path.getsize(output_path)
        print(f"Arquivo Parquet com compressão {compression if compression else 'sem compressão'} criado em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho do arquivo: {tamanho_arquivo} bytes")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
    except Exception as e:
        print(f"Ocorreu um erro ao escrever o Parquet: {e}")

def main():
    """Função principal para executar a aplicação de compressão."""
    input_parquet_path = '../../databases/parquet/titanic.parquet' # Caminho para o Parquet ORIGINAL
    output_dir = 'dataset/compression'
    os.makedirs(output_dir, exist_ok=True)

    # Verifica se o arquivo parquet original existe
    if not os.path.exists(input_parquet_path):
        print(f"Erro: Arquivo Parquet original '{input_parquet_path}' não encontrado. Certifique-se de que ele exista.")
        return # Encerra a execução se o arquivo não existe

    compressions = {
        'sem_compressao': None,
        'snappy': 'snappy',
        'gzip': 'gzip',
        'brotli': 'brotli',
        'zstd': 'zstd'
    }

    for nome, compressao in compressions.items():
        output_parquet_path = os.path.join(output_dir, f'titanic_{nome}.parquet')
        criar_parquet_com_compressao(input_parquet_path, output_parquet_path, compression=compressao)


if __name__ == "__main__":
    main()

In [ ]:
def comprimir_parquet_snappy(input_path, output_path):
    """
    Comprime um arquivo Parquet usando Snappy.

    Args:
        input_path (str): Caminho para o arquivo Parquet de entrada.
        output_path (str): Caminho para o arquivo Parquet de saída (com compressão Snappy).
    """
    try:
        start_time = time.time()

        # Ler o arquivo Parquet usando PyArrow
        table = pq.read_table(input_path)

        # Escrever o arquivo Parquet com compressão Snappy
        pq.write_table(table, output_path, compression='snappy')

        end_time = time.time()

        tamanho_original = os.path.getsize(input_path)
        tamanho_comprimido = os.path.getsize(output_path)
        taxa_compressao = (tamanho_original - tamanho_comprimido) / tamanho_original * 100

        print(f"Arquivo comprimido com Snappy em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho original: {tamanho_original} bytes")
        print(f"Tamanho comprimido: {tamanho_comprimido} bytes")
        print(f"Taxa de compressão: {taxa_compressao:.2f}%")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
    except Exception as e:
        print(f"Ocorreu um erro durante a compressão: {e}")

def main():
    """Função principal."""
    input_parquet = '../../databases/parquet/titanic.parquet'  # Substitua pelo caminho do seu arquivo Parquet
    output_parquet_snappy = 'dataset/titanic_snappy.parquet' # Substitua pelo caminho de saida desejado
    output_dir = 'dataset'
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_parquet):
        print(f"Erro: Arquivo Parquet original '{input_parquet}' não encontrado.")
        return

    comprimir_parquet_snappy(input_parquet, output_parquet_snappy)

if __name__ == "__main__":
    main()